![atlas](atlas4.png)

A fire atlas is a configuration convention for geospatial assets related to community fire planning and response together with a configuration for ways to instantiate, edit, and manage those assets.

A stewardship atlas is a data set, a confuration for storing, processing, and sharing that data set, and a set of implementions to do so.

In [1]:
# Boring Imports
import sys, os, subprocess, time, json, string, datetime, random, math
sys.path.insert(0, "/root/stewardship_atlas/python")

# our Imports
import json_config
import dataswale_geojson
import deltas_geojson as deltas
import versioning
import vector_inlets
import raster_inlets
import outlets
import atlas
import utils
import eddies

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# Create a new atlas from GeoJSON

In [7]:
gj = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {"name": "westport", "logo": "https://cdn.prod.website-files.com/648ea312c78f6d7a5ca19989/64c2933be813b5edc51b9063_WFDlogo_DS.png"},
      "geometry": {
        "coordinates": [ utils.bbox_to_polygon( {
        "north": 39.89595306214338,
        "south": 39.53857292439969,
        "east": -123.67763908625702,
        "west": -123.85333388805361
    }
)],
        "type": "Polygon"
      }
    }
  ]
}


# c = atlas.create(feature_collection=gj)

layers = json.load(open("../configuration/wvfd_layers.json"))
assets = json.load(open("../configuration/wvfd_assets.json"))

c = atlas.create(feature_collection=gj, layers=layers, assets=assets)

Updating password for user internal
Updating password for user admin
Updating password for user admin
Updating password for user internal
Updating password for user admin
Updating password for user admin
Updating password for user admin
Updating password for user internal
Updating password for user admin
2025-08-06 20:50:47,630 - atlas - INFO - Storing initial feature collection in {p /'layers' / 'regions'}...
2025-08-06 20:50:47,632 - atlas - INFO - built a config for westport.


### Or use an existing swale and config:

In [ ]:
c = json.load(open("/root/swales/westport/staging/atlas_config.json"))


# New School

In [3]:
materializers =  outlets.asset_methods | eddies.asset_methods | vector_inlets.asset_methods
#atlas.materialize(materializers, c, 'html')
materializers

{'html': <function outlets.outlet_html(config, outlet_name)>,
 'runbook': <function outlets.outlet_runbook(config, outlet_name, skips=[], start_at=0, limit=0)>,
 'webmap': <function outlets.outlet_webmap(config, name)>,
 'webedit': <function outlets.outlet_webmap_edit(config: dict, name: str)>,
 'sqlquery': <function outlets.outlet_sqlquery(config: dict, outlet_name: str)>,
 'gdal_contours': <function eddies.contours_gdal(config: Dict[str, Any], eddy_name: str)>,
 'derived_hillshade': <function eddies.hillshade_gdal(config: Dict[str, Any], eddy_name: str)>,
 'annotate_spatial': <function eddies.delta_annotate_spatial_duckdb(config: Dict[str, Any], layer_name: str, delta_name: str, anno_type: str = 'deltas', anno_in_path: pathlib.Path = None, updated_properties: List[str] = [])>,
 'overture_duckdb': <function vector_inlets.overture_duckdb(config=None, name=None, delta_queue=<module 'deltas_geojson' from '/root/stewardship_atlas/python/deltas_geojson.py'>, quick=False)>,
 'local_ogr': <f

In [ ]:
[l['name'] for l in c['dataswale']['layers']]

In [ ]:
c['assets']['runbook']

In [ ]:
 regions_path = versioning.atlas_path(c, "layers") / "regions" / "regions.geojson"

In [ ]:
outlets.regions_from_geojson(regions_path, start_at=0, limit=0)

### Elevation and Hillshade rasters and vectors

In [ ]:
raster_inlets.url_raster(config=c, name="dem", delta_queue=deltas)

In [ ]:
dataswale_geojson.refresh_raster_layer(c, 'elevation', deltas.apply_deltas)

In [ ]:
dataswale_geojson.eddy(c, 'derived_hillshade')

In [ ]:
dataswale_geojson.eddy(c, 'gdal_contours')

### Core vector layers

In [4]:
# vector_inlets.overture_duckdb(config=c, name="public_roads", delta_queue=deltas)
atlas.materialize(materializers, c, 'public_roads')
dataswale_geojson.refresh_vector_layer(c, 'roads')

atlas.materialize(materializers, c, 'public_creeks')
dataswale_geojson.refresh_vector_layer(c, 'creeks', deltas.apply_deltas)

2025-08-06 17:11:26,261 - vector_inlets - INFO - Fetching Overture data with query: SET s3_region='us-west-2';
SELECT
  * EXCLUDE geometry, names.primary AS primary_name, ST_AsGeoJSON(ST_GeomFromWKB(geometry)) as geom
FROM
  read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=transportation/type=segment/*', filename=true, hive_partitioning=1)
WHERE
  bbox.xmin > -123.85333388805361
  AND bbox.xmax < -123.67763908625702
  AND bbox.ymin > 39.53857292439969
  AND bbox.ymax < 39.89595306214338
  AND
  LIST_CONTAINS(['primary', 'secondary', 'tertiary', 'service', 'residential'], class)

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


2025-08-06 17:14:42,289 - deltas_geojson - INFO - Wrote 269 features to /root/swales/westport/staging/deltas/roads/public_roads__20250806_171441__create.geojson
2025-08-06 17:14:42,338 - utils - INFO - Altering GeoJSON in /root/swales/westport/staging/deltas/roads/public_roads__20250806_171441__create.geojson with {'canonicalize': [{'to': 'name', 'from': ['primary_name', 'FULLRDNAME'], 'remove_prefix': []}], 'vector_width': {'attribute': 'class', 'map': {'primary': 15, 'secondary': 11, 'tertiary': 9}, 'default': 7}}.
2025-08-06 17:14:42,985 - deltas_geojson - INFO - Starting Apply Deltas: /root/swales/westport/staging/deltas/roads: [PosixPath('/root/swales/westport/staging/deltas/roads/public_roads__20250806_171441__create.geojson')] -> /root/swales/westport/staging/deltas/roads/work
2025-08-06 17:14:42,988 - deltas_geojson - INFO - delta file /root/swales/westport/staging/deltas/roads/public_roads__20250806_171441__create.geojson
2025-08-06 17:14:42,990 - deltas_geojson - INFO - delta

Running ogr2ogr with args: ['ogr2ogr', '-f', 'GeoJSON', '-t_srs', 'EPSG:4269', '-spat', '-123.85333388805361', '39.53857292439969', '-123.67763908625702', '39.89595306214338', '-spat_srs', 'EPSG:4269', '/root/swales/westport/staging/deltas/creeks/public_creeks__20250806_171449__create.geojson', '/root/swales/westport/staging/local/NHD_H_California_State_GPKG.gpkg', 'NHDFlowline']


2025-08-06 17:15:02,428 - utils - INFO - Altering GeoJSON in /root/swales/westport/staging/deltas/creeks/public_creeks__20250806_171449__create.geojson with {'canonicalize': [{'to': 'name', 'from': ['gnis_name'], 'remove_prefix': []}], 'vector_width': {'attribute': 'fcode_description', 'map': {'Stream/River: Hydrographic Category = Ephemeral': 2, 'Stream/River: Hydrographic Category = Perennial': 3, 'Stream/River: Hydrographic Category = Intermittent': 1, 'Artificial Path': 1}, 'default': 1}}.
2025-08-06 17:15:06,901 - deltas_geojson - INFO - Starting Apply Deltas: /root/swales/westport/staging/deltas/creeks: [PosixPath('/root/swales/westport/staging/deltas/creeks/public_creeks__20250806_171449__create.geojson'), PosixPath('/root/swales/westport/staging/deltas/creeks/public_creeks__20250610_224927__create.geojson')] -> /root/swales/westport/staging/deltas/creeks/work
2025-08-06 17:15:06,902 - deltas_geojson - INFO - delta file /root/swales/westport/staging/deltas/creeks/public_creeks__

PosixPath('/root/swales/westport/staging/layers/creeks/creeks.geojson')

In [5]:
# vector_inlets.local_ogr(config=c, name="public_creeks", delta_queue=deltas)
# vector_inlets.local_ogr(config=c, name="local_milemarkers", delta_queue=deltas)
atlas.materialize(materializers, c, 'local_milemarkers')
dataswale_geojson.refresh_vector_layer(c, 'milemarkers', deltas.apply_deltas)

Running ogr2ogr with args: ['ogr2ogr', '-f', 'GeoJSON', '-t_srs', 'EPSG:4269', '-spat', '-123.85333388805361', '39.53857292439969', '-123.67763908625702', '39.89595306214338', '-spat_srs', 'EPSG:4269', '/root/swales/westport/staging/deltas/milemarkers/local_milemarkers__20250806_171555__create.geojson', '/root/swales/westport/staging/local/postmiles_tenth.geojson']


2025-08-06 17:17:39,778 - utils - INFO - Altering GeoJSON in /root/swales/westport/staging/deltas/milemarkers/local_milemarkers__20250806_171555__create.geojson with {'canonicalize': [{'to': 'name', 'from': ['PMc'], 'remove_prefix': []}], 'vector_width': {'default': 3}, 'require': [['endswith', ['0.0', '5.0']], ['equals', [['91.0', 'Callbox #911 mm 91.08'], ['89.6', 'Callbox #896 mm 89.63'], ['87.1', 'Callbox #878 mm 87.15'], ['85.7', 'Callbox #860 mm 85.74'], ['84.0', 'Callbox #840 mm 84.0'], ['81.6', 'Callbox #817 mm 81.62'], ['79.1', 'Callbox #790 mm 79.12'], ['76.5', 'Callbox #768 mm 76.57'], ['71.8', 'Callbox #714 mm 71.84']]]]}.
2025-08-06 17:17:40,176 - deltas_geojson - INFO - Starting Apply Deltas: /root/swales/westport/staging/deltas/milemarkers: [PosixPath('/root/swales/westport/staging/deltas/milemarkers/local_milemarkers__20250806_171555__create.geojson')] -> /root/swales/westport/staging/deltas/milemarkers/work
2025-08-06 17:17:40,179 - deltas_geojson - INFO - delta file /

PosixPath('/root/swales/westport/staging/layers/milemarkers/milemarkers.geojson')

### Not currently in use...

In [ ]:
#vector_inlets.local_ogr(config=c, name="local_turnouts", delta_queue=deltas)
atlas.materialize(materializers, c, 'local_turnouts')
dataswale_geojson.refresh_vector_layer(c, 'turnouts')

# vector_inlets.local_ogr(config=c, name="internal_roads", delta_queue=deltas)
atlas.materialize(materializers, c, 'internal_roads')
dataswale_geojson.refresh_vector_layer(c, 'internal_roads', deltas.apply_deltas)

### Buildings, Parcels, and Addresses

In [ ]:
vector_inlets.overture_duckdb(config=c, name="public_buildings", delta_queue=deltas)

dataswale_geojson.refresh_vector_layer(c, 'buildings', deltas.apply_deltas)

#### First load parcel layer from OpenAddresses

In [ ]:
vector_inlets.local_ogr(config=c, name="oa_parcels", delta_queue=deltas)

dataswale_geojson.refresh_vector_layer(c, 'parcels', deltas.apply_deltas)

#### Then get addresses from Oveture - as a Delta to annotate parcels

In [ ]:
vector_inlets.overture_duckdb(config=c, name="public_addresses", delta_queue=deltas)

eddies.delta_annotate_spatial_duckdb(c, "parcels", "public_addresses__20250612_223703__create")

In [ ]:
# No need to run this since the above annoation updates the layer
dataswale_geojson.refresh_vector_layer(c, 'parcels', deltas.apply_deltas)

#### Now use parcel layer as annotation for Buildings

In [ ]:
eddies.delta_annotate_spatial_duckdb(c, "buildings", "parcels", anno_type="layers")

### POI etc

In [8]:
vector_inlets.local_ogr(config=c, name="local_hydrants", delta_queue=deltas)

dataswale_geojson.refresh_vector_layer(c, 'hydrants', deltas.apply_deltas_overwrite)

Running ogr2ogr with args: ['ogr2ogr', '-f', 'GeoJSON', '-t_srs', 'EPSG:4269', '/root/swales/westport/staging/deltas/hydrants/local_hydrants__20250806_205105__create.geojson', '/root/swales/westport/staging/local/westport_hydrants.geojson']


2025-08-06 20:51:06,024 - utils - INFO - Altering GeoJSON in /root/swales/westport/staging/deltas/hydrants/local_hydrants__20250806_205105__create.geojson with {'canonicalize': [{'to': 'name', 'from': ['label'], 'default': 'hydrant'}], 'vector_width': {'default': 5}}.
2025-08-06 20:51:06,029 - deltas_geojson - INFO - Starting Apply Deltas: /root/swales/westport/staging/deltas/hydrants: [PosixPath('/root/swales/westport/staging/deltas/hydrants/local_hydrants__20250806_171742__create.geojson'), PosixPath('/root/swales/westport/staging/deltas/hydrants/local_hydrants__20250806_205105__create.geojson')] -> /root/swales/westport/staging/deltas/hydrants/work
2025-08-06 20:51:06,030 - deltas_geojson - INFO - delta file /root/swales/westport/staging/deltas/hydrants/local_hydrants__20250806_171742__create.geojson
2025-08-06 20:51:06,034 - deltas_geojson - INFO - delta file /root/swales/westport/staging/deltas/hydrants/local_hydrants__20250806_171742__create.geojson: local_hydrants @ 20250806_171

PosixPath('/root/swales/westport/staging/layers/hydrants/hydrants.geojson')

In [9]:
vector_inlets.local_ogr(config=c, name="local_helilandings", delta_queue=deltas)

dataswale_geojson.refresh_vector_layer(c, 'helilandings', deltas.apply_deltas_overwrite)

Running ogr2ogr with args: ['ogr2ogr', '-f', 'GeoJSON', '-t_srs', 'EPSG:4269', '/root/swales/westport/staging/deltas/helilandings/local_helilandings__20250806_205117__create.geojson', '/root/swales/westport/staging/local/westport_helilandings.geojson']


2025-08-06 20:51:18,268 - utils - INFO - Altering GeoJSON in /root/swales/westport/staging/deltas/helilandings/local_helilandings__20250806_205117__create.geojson with {'vector_width': {'default': 10}}.
2025-08-06 20:51:18,420 - deltas_geojson - INFO - Starting Apply Deltas: /root/swales/westport/staging/deltas/helilandings: [PosixPath('/root/swales/westport/staging/deltas/helilandings/local_helilandings__20250806_205117__create.geojson'), PosixPath('/root/swales/westport/staging/deltas/helilandings/local_helilandings__20250610_233729__create.geojson')] -> /root/swales/westport/staging/deltas/helilandings/work
2025-08-06 20:51:18,421 - deltas_geojson - INFO - delta file /root/swales/westport/staging/deltas/helilandings/local_helilandings__20250806_205117__create.geojson
2025-08-06 20:51:18,424 - deltas_geojson - INFO - delta file /root/swales/westport/staging/deltas/helilandings/local_helilandings__20250806_205117__create.geojson: local_helilandings @ 20250806_205117 -> create
2025-08-

PosixPath('/root/swales/westport/staging/layers/helilandings/helilandings.geojson')

## Outlets

In [14]:
# outlets.outlet_runbook(c, 'runbook', skips=['region_maps'])
atlas.materialize(materializers, c, 'runbook')

2025-08-06 20:57:43,701 - outlets - INFO - Extracting region hwy_20_to_county_linee of vector layer tsunami to /root/swales/westport/staging/outlets/runbook/tsunami_hwy_20_to_county_linee.geojson.
2025-08-06 20:58:49,464 - outlets - INFO - Processed /root/swales/westport/staging/outlets/runbook/tsunami_hwy_20_to_county_linee.geojson -> /root/swales/westport/staging/outlets/runbook/tsunami_hwy_20_to_county_linee_processed.geojson
2025-08-06 20:58:49,537 - outlets - INFO - Reusing vector file at: /root/swales/westport/staging/outlets/runbook/tsunami_ocean_meadows.geojson
2025-08-06 20:58:50,106 - outlets - INFO - Processed /root/swales/westport/staging/outlets/runbook/tsunami_ocean_meadows.geojson -> /root/swales/westport/staging/outlets/runbook/tsunami_ocean_meadows_processed.geojson
2025-08-06 20:58:50,107 - outlets - INFO - Reusing vector file at: /root/swales/westport/staging/outlets/runbook/tsunami_seaside_beach.geojson
2025-08-06 20:58:50,503 - outlets - INFO - Processed /root/swal

2025-08-06 20:59:20,101 - outlets - INFO - Extracting region lz_overview84_81 of vector layer tsunami to /root/swales/westport/staging/outlets/runbook/tsunami_lz_overview84_81.geojson.
2025-08-06 20:59:41,296 - outlets - INFO - Processed /root/swales/westport/staging/outlets/runbook/tsunami_lz_overview84_81.geojson -> /root/swales/westport/staging/outlets/runbook/tsunami_lz_overview84_81_processed.geojson
2025-08-06 20:59:41,298 - outlets - INFO - Extracting region callboxovery of vector layer tsunami to /root/swales/westport/staging/outlets/runbook/tsunami_callboxovery.geojson.
2025-08-06 21:00:35,065 - outlets - INFO - Processed /root/swales/westport/staging/outlets/runbook/tsunami_callboxovery.geojson -> /root/swales/westport/staging/outlets/runbook/tsunami_callboxovery_processed.geojson
2025-08-06 21:00:35,066 - outlets - INFO - Extracting region ao of vector layer tsunami to /root/swales/westport/staging/outlets/runbook/tsunami_ao.geojson.
2025-08-06 21:01:39,857 - outlets - INFO 

2025-08-06 21:08:10,740 - outlets - INFO - Extracting region seaside_beach of raster layer lidar_basemap to /root/swales/westport/staging/outlets/runbook/lidar_basemap_seaside_beach.tiff.
2025-08-06 21:08:25,860 - outlets - INFO - Processing raster region: seaside_beach: {'name': 'seaside_beach', 'caption': 'Seaside Beach', 'text': 'Region 2', 'bbox': {'west': -123.7709884652784, 'east': -123.76162482276492, 'north': 39.56411775236229, 'south': 39.557299914789894}, 'neighbors': {'prev': 'ocean_meadows', 'next': 'region_3'}, 'vectors': [[{'name': 'tsunami', 'geometry_type': 'polygon', 'fill_color': [0, 0, 250], 'fill_opacity': 0.3}, '/root/swales/westport/staging/outlets/runbook/tsunami_seaside_beach_processed.geojson'], [{'name': 'contours', 'geometry_type': 'linestring', 'color': [50, 200, 50]}, '/root/swales/westport/staging/outlets/runbook/contours_seaside_beach_processed.geojson']], 'raster': [{'name': 'lidar_basemap', 'geometry_type': 'raster'}, '/root/swales/westport/staging/outl

2025-08-06 21:10:14,100 - outlets - INFO - Extracting region branscomb_mm79 of raster layer lidar_basemap to /root/swales/westport/staging/outlets/runbook/lidar_basemap_branscomb_mm79.tiff.
2025-08-06 21:10:29,697 - outlets - INFO - Processing raster region: branscomb_mm79: {'name': 'branscomb_mm79', 'caption': 'Branscomb MM79', 'text': 'Region 10', 'bbox': {'west': -123.79028221759613, 'east': -123.73977357766051, 'north': 39.665392927025685, 'south': 39.640961999880325}, 'neighbors': {'prev': 'wages_creek', 'next': 'dehaven'}, 'vectors': [[{'name': 'tsunami', 'geometry_type': 'polygon', 'fill_color': [0, 0, 250], 'fill_opacity': 0.3}, '/root/swales/westport/staging/outlets/runbook/tsunami_branscomb_mm79_processed.geojson'], [{'name': 'contours', 'geometry_type': 'linestring', 'color': [50, 200, 50]}, '/root/swales/westport/staging/outlets/runbook/contours_branscomb_mm79_processed.geojson']], 'raster': [{'name': 'lidar_basemap', 'geometry_type': 'raster'}, '/root/swales/westport/stagi

2025-08-06 21:12:13,312 - outlets - INFO - Extracting region harcy_creek of raster layer lidar_basemap to /root/swales/westport/staging/outlets/runbook/lidar_basemap_harcy_creek.tiff.
2025-08-06 21:12:28,672 - outlets - INFO - Processing raster region: harcy_creek: {'name': 'harcy_creek', 'caption': 'Harcy Creek', 'text': 'Region 18', 'bbox': {'west': -123.81085397278572, 'east': -123.79704459821346, 'north': 39.71597622662519, 'south': 39.70824563608746}, 'neighbors': {'prev': 'hwy1_mm_77_to_83', 'next': 'hardy_to_rockport'}, 'vectors': [[{'name': 'tsunami', 'geometry_type': 'polygon', 'fill_color': [0, 0, 250], 'fill_opacity': 0.3}, '/root/swales/westport/staging/outlets/runbook/tsunami_harcy_creek_processed.geojson'], [{'name': 'contours', 'geometry_type': 'linestring', 'color': [50, 200, 50]}, '/root/swales/westport/staging/outlets/runbook/contours_harcy_creek_processed.geojson']], 'raster': [{'name': 'lidar_basemap', 'geometry_type': 'raster'}, '/root/swales/westport/staging/outle

...

2025-08-06 23:19:12,508 - outlets - INFO - Blended raster using percent: 50 [286.4876620769501]
2025-08-06 23:20:10,032 - outlets - INFO - callboxovery : tsunami [344.0117757320404]
2025-08-06 23:20:57,144 - outlets - INFO - callboxovery : contours [391.1236295700073]
2025-08-06 23:21:09,714 - outlets - INFO - callboxovery : roads [403.69352769851685]
2025-08-06 23:21:53,308 - outlets - INFO - callboxovery : creeks [447.28740429878235]
2025-08-06 23:23:48,266 - outlets - INFO - callboxovery : buildings [562.2464001178741]
2025-08-06 23:24:10,751 - outlets - INFO - Building map for region: ao  [8835.923674821854] with wtf config: {'name': 'ao', 'caption': 'AO', 'text': 'Region 25', 'bbox': {'west': -123.85333388805361, 'east': -123.67763908625702, 'north': 39.89595306214338, 'south': 39.53857292439969}, 'neighbors': {'prev': 'callboxovery', 'next': 'hwy_20_to_county_linee'}, 'vectors': [[{'name': 'tsunami', 'geometry_type': 'polygon', 'fill_color': [0, 0, 250], 'fill_opacity': 0.3}, '/r

making map image for {'name': 'ao', 'caption': 'AO', 'text': 'Region 25', 'bbox': {'west': -123.85333388805361, 'east': -123.67763908625702, 'north': 39.89595306214338, 'south': 39.53857292439969}, 'neighbors': {'prev': 'callboxovery', 'next': 'hwy_20_to_county_linee'}, 'vectors': [[{'name': 'tsunami', 'geometry_type': 'polygon', 'fill_color': [0, 0, 250], 'fill_opacity': 0.3}, '/root/swales/westport/staging/outlets/runbook/tsunami_ao_processed.geojson'], [{'name': 'contours', 'geometry_type': 'linestring', 'color': [50, 200, 50]}, '/root/swales/westport/staging/outlets/runbook/contours_ao_processed.geojson'], [{'name': 'roads', 'geometry_type': 'linestring', 'color': [100, 55, 50], 'add_labels': True, 'interaction': 'interface', 'vector_width': True, 'editable_columns': [{'name': 'name', 'type': 'string', 'default': 'Road'}, {'name': 'STREETTYPE', 'type': 'radio', 'values': [{'STREETTYPE': 'RD', 'vector_width': 15}, {'STREETTYPE': 'ST', 'vector_width': 13}, {'STREETTYPE': 'Other', 've

2025-08-06 23:31:20,900 - outlets - INFO - Blended raster using percent: 50 [430.00301861763]
2025-08-06 23:32:43,384 - outlets - INFO - ao : tsunami [512.4873313903809]
2025-08-06 23:33:49,059 - outlets - INFO - ao : contours [578.162348985672]
2025-08-06 23:34:11,220 - outlets - INFO - ao : roads [600.3232173919678]
2025-08-06 23:34:56,262 - outlets - INFO - ao : creeks [645.36501288414]
2025-08-06 23:36:47,068 - outlets - INFO - ao : buildings [756.1708636283875]
2025-08-06 23:37:06,426 - outlets - INFO - Writing region output to: /root/swales/westport/staging/outlets/runbook/hwy_20_to_county_linee.html  [9611.599455356598]
2025-08-06 23:37:06,428 - outlets - INFO - Writing region output to: /root/swales/westport/staging/outlets/runbook/ocean_meadows.html  [9611.600648641586]
2025-08-06 23:37:06,428 - outlets - INFO - Writing region output to: /root/swales/westport/staging/outlets/runbook/seaside_beach.html  [9611.601497888565]
2025-08-06 23:37:06,429 - outlets - INFO - Writing regi

b''


[{'name': 'hwy_20_to_county_linee',
  'caption': 'Hwy 20 to county linee',
  'text': 'Region 0',
  'bbox': {'west': -124.03994509157229,
   'east': -123.31737226889331,
   'north': 40.00654369459244,
   'south': 39.344401456064766},
  'neighbors': {'prev': 'ao', 'next': 'ocean_meadows'},
  'vectors': [[{'name': 'tsunami',
     'geometry_type': 'polygon',
     'fill_color': [0, 0, 250],
     'fill_opacity': 0.3},
    '/root/swales/westport/staging/outlets/runbook/tsunami_hwy_20_to_county_linee_processed.geojson'],
   [{'name': 'contours',
     'geometry_type': 'linestring',
     'color': [50, 200, 50]},
    '/root/swales/westport/staging/outlets/runbook/contours_hwy_20_to_county_linee_processed.geojson'],
   [{'name': 'roads',
     'geometry_type': 'linestring',
     'color': [100, 55, 50],
     'add_labels': True,
     'interaction': 'interface',
     'vector_width': True,
     'editable_columns': [{'name': 'name',
       'type': 'string',
       'default': 'Road'},
      {'name': 'STR

In [10]:
outlets.outlet_webmap(c, 'webmap')

2025-08-06 20:51:21,862 - outlets - INFO - In webedit, got Outlet Conf: {'type': 'outlet', 'name': 'webmap', 'in_layers': ['basemap', 'tsunami', 'parcels', 'contours', 'roads', 'milemarkers', 'creeks', 'buildings', 'natural'], 'config_def': 'webmap', 'access': ['internal', 'admin'], 'config': {'asset_type': 'outlet', 'name': 'webmap', 'fetch_type': 'webmap', 'interaction': 'interface', 'attribution': {'url': 'https://portal.opentopography.org/api/terms', 'description': "Open Topography's Global DEM", 'license': 'https://portal.opentopography.org/api/terms', 'citation': 'https://portal.opentopography.org/api/terms'}}}
2025-08-06 20:51:21,865 - outlets - INFO - Generating basemap: /root/swales/westport/staging/layers/basemap/basemap.jpg.
Warning 6: JPEG driver doesn't support data type Float32. Only eight and twelve bit bands supported (Mk1 libjpeg).

2025-08-06 20:51:23,224 - outlets - INFO - Creating webmap HTML in /root/swales/westport/staging/outlets/webmap/index.html.


PosixPath('/root/swales/westport/staging/outlets/webmap/index.html')

In [ ]:
outlets.outlet_webmap_edit(c, 'webedit')

In [ ]:
c['assets']['runbook']['in_layers']

In [ ]:
outlets.outlet_runbook(c, 'runbook', start_at=1, limit=2) #, skips=['region_maps'])

In [ ]:
webedit_dir = versioning.atlas_path(c, "outlets") / 'webedit'
#subprocess.run(['ls', '-lha', '../templates/js/'])
subprocess.check_output(['cp', '-r', '../templates/js/', str(webedit_dir / "js")])

In [ ]:
c['assets']['webmap']

In [ ]:
c['assets']['internal_webmap']

In [12]:
c['spreadsheets'] = {}

In [13]:
outlets.outlet_html(c, 'html')

2025-08-06 20:53:34,671 - outlets - INFO - Generated Internal interfaces: [{'type': 'outlet', 'name': 'webmap', 'in_layers': ['basemap', 'tsunami', 'parcels', 'contours', 'roads', 'milemarkers', 'creeks', 'buildings', 'natural'], 'config_def': 'webmap', 'access': ['internal', 'admin'], 'config': {'asset_type': 'outlet', 'name': 'webmap', 'fetch_type': 'webmap', 'interaction': 'interface', 'attribution': {'url': 'https://portal.opentopography.org/api/terms', 'description': "Open Topography's Global DEM", 'license': 'https://portal.opentopography.org/api/terms', 'citation': 'https://portal.opentopography.org/api/terms'}}}, {'type': 'outlet', 'name': 'runbook', 'in_layers': ['lidar_basemap', 'tsunami', 'contours', 'roads', 'creeks', 'buildings', 'campgrounds', 'helilandings', 'milemarkers', 'hydrants'], 'access': ['internal', 'admin'], 'config_def': 'runbook', 'blend_percent': 50, 'regions': [{'bbox': {'east': -121.19978063408502, 'west': -121.20391109005016, 'south': 39.23863749098538, '

PosixPath('/root/swales/westport/staging/outlets/html')

In [ ]:
[
        an for an,ac in c['assets'].items()
        if ac['type'] == 'outlet'
        and ac.get('config',{}).get('interaction') == 'interface'
        and ac.get('access',['public']).count('admin') > 0
        # and ac.get('interaction') == 'interface'                                                                                                         
        # and ac.get('access') in ('admin', 'internal', 'public')                                                                                          
    ]


In [ ]:
outlets.outlet_sqlquery(c, 'sqlquery')

In [ ]:
outlets.sql_query(c, 'sqldb', "SELECT * FROM roads;")

In [ ]:
def materialize(c, n):
    materializer_name = c['assets'][n]['config']['fetch_type']
    return outlets.MATERIALIZERS[materializer_name](c,n)

In [ ]:
materializers =  outlets.asset_methods | eddies.asset_methods | vector_inlets.asset_methods
atlas.materialize(materializers, c, 'html')
#materializers

# OLD Inlets

In [ ]:
g=atlas.asset_materialize(ac, dc, ac['assets']['addresses'])

In [ ]:
g=atlas.asset_materialize(ac, dc, ac['assets']['raw_buildings'])

In [ ]:
g=atlas.asset_materialize(ac, dc, ac['assets']['buildings'])

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['raw_creeky'])

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['creeky'])

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['milemarkers'])

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['campsites'])
# 3 B

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['poi_deltas'], version_string='stage')

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['poi'], version_string='stage')

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['tsunami'])

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['dem'])

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['contours'])

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['hillshade'])
#atlas.asset_materialize(ac, dc, ac['assets']['landscape'])

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['roads_delta'])

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['overture_roads'])

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['private_roads'])

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['natural'])

In [ ]:
atlas.asset_materialize(ac, dc, ac['assets']['hydrants'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['notes_delta'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['notes'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['raw_ponds'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['ponds'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['gates'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['helilanding'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['campsites'])

## Outlets

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['html'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['gazetteer'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['tiff'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['geopdf'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['mbtile'])

In [ ]:
full_regions = ac['assets']['runbook']['regions']

In [ ]:
ac['assets']['runbook']['regions'][0]

In [ ]:
ac['assets']['runbook']['regions'][0]['skip'] = []

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['public_runbook'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['runbook'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['webmap'])

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['webedit'])

# Messing About

In [ ]:
s="""{"version_string": "staging", "versions": ["published"], "logo": "/local/scs-smallgrass1.png", "swaleName": "westport", "consoleType": "INTERNAL", "interfaces": [{"type": "outlet", "in_layers": ["basemap", "parcels", "contours", "roads", "internal_roads", "turnouts", "milemarkers", "creeks", "buildings", "helilandings", "hydran\
ts"], "config_def": "webmap", "access": ["internal", "admin"], "config": {"asset_type": "outlet", "name": "webmap", "fetch_type": "webmap", "interaction": "interface", "attribu\
tion": {"url": "https://portal.opentopography.org/api/terms", "description": "Open Topography's Global DEM", "license": "https://portal.opentopography.org/api/terms", "citation\
": "https://portal.opentopography.org/api/terms"}}}, {"type": "outlet", "name": "runbook", "in_layers": ["lidar_basemap", "tsunami", "contours", "roads", "creeks", "buildings",\
 "campgrounds", "helilandings", "milemarkers", "hydrants"], "access": ["internal", "admin"], "config_def": "runbook", "regions": [{"bbox": {"east": -121.19978063408502, "west":\
 -121.20391109005016, "south": 39.23863749098538, "north": 39.24416744687048}, "name": "RockLoop", "caption": "Double loop with Ginger and big rocks.", "vectors": [], "raster":\
 "", "text": "Hwy20 to the County Lnie"}, {"bbox": {"east": -121.19978063408502, "west": -121.20391109005016, "south": 39.23863749098538, "north": 39.24416744687048}, "name": "\
GolfCourses", "caption": "Golf Courses", "vectors": [], "raster": "", "text": "Golf Courses"}], "config": {"asset_type": "outlet", "layers": ["hillshade", "contours", "creeks",\
 "tsunami", "buildings", "helilanding", "overture_roads"], "fetch_type": "runbook", "interaction": "interface", "data_type": "html", "outpath_template": "/runbook/{name}_page_{\
i}.html", "page_name_template": "{name}_page_{i}", "attribution": {"url": "https://www.scvfd.fireatlas.org", "description": "Runbook of the atlas, a linked set of specific subm\
ap/diagram pages.", "license": "https://www.scvfd.fireatlas.org", "citation": "https://www.scvfd.fireatlas.org"}}}], "downloads": [], "useCases": [{"name": "Firefighter", "case\
s": ["Download Avenza version", "Share a QR Code for Avenza", "Mark an Incident", "Mark a POI"]}, {"name": "GIS Practitioner", "cases": ["Download Layer GeoJSON", "Download Geo\
PKG", "Add a layer as GeoJSON"]}, {"name": "Administrator", "cases": ["Go to Admin interface", "Switch Version"]}], "layers": []}"""
print(json.dumps(json.loads(s)['interfaces'], indent=2))

In [ ]:
[].count('d')

In [ ]:
def canonicalize_name(s):
    return "_".join(s.lower().split()).strip()

In [ ]:
canonicalize_name("    h    IU H8  hdh")

In [ ]:
#json.dump(hmm, open("newregions.json", "w"))
regions = json.load( open("newregions.json"))

In [ ]:
utils.geojson_to_bbox([
            [
              -121.19978063408502,
              39.24416744687048
            ],
            [
              -121.20391109005016,
              39.24416744687048
            ],
            [
              -121.20391109005016,
              39.23863749098538
            ],
            [
              -121.19978063408502,
              39.23863749098538
            ],
            [
              -121.19978063408502,
              39.24416744687048
            ]
          ])

In [ ]:
regions[9]['vectors'][-1][0]=ac['assets']['overture_roads']


In [ ]:
atlas_outlets.build_region_map('/root/data/', 'westport', 'stage', 'runbook', regions[9])

In [ ]:
"DFDf".lower()

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['webmap_public'])

In [ ]:
z=json.load(open("zones.geojson"))

In [ ]:
z

In [ ]:
ac['assets'].keys()

In [ ]:
regions[1]['vectors'][0][0]['color']=[0,255,0]

In [ ]:
del(regions[1]['vectors'][0])

In [ ]:
regions[0][1]['raster'] = [ac['assets']['hillshade'], '/root/data/scvfd/stage/staging/hillshade_1_A.tiff']

In [ ]:
atlas_outlets.build_region_map('/root/data/', 'westport', 'stage', '', regions[1])

In [ ]:
#ht5#.format(ea=ac['assets']['creeky'],**ac['assets']['creeky'])
ht1.format(name=ac['assets']['creeky']['name'])
#ac['assets']['creeky']

In [ ]:
import planetary_computer

In [ ]:
dir(planetary_computer)

In [ ]:
for i in range(10) if i %% 3:
    print(i)
    

In [ ]:
j = json.load(open('/root/data/scvfd/staging/roads.geojson'))

from collections import Counter
c=Counter()
for f in j['features']:
    
    c.update(f['properties'].keys())

regions = atlas.asset_materialize(ac, dc, ac['assets']['gazetteer'])

import atlas_outlets
atlas_outlets.export_region_layer_raster('/root/data/', 'scvfd', 'hillshade', regions[0])

In [ ]:
outlets.sql_query(c, 'sqldb', 
                  f"""SELECT column_name 
                  FROM information_schema.columns 
                  WHERE table_name = 'creeks';""")

In [ ]:
regions = atlas_outlets.generate_regions(dc, ac['assets']['gazetteer'])

region_json = json.dump(regions,open('regions_scvfd.json', 'w'))

import atlas_eddies
atlas_eddies.generate_contours('/root/data/scvfd/stage/staging/dem.tiff', '/root/data/scvfd/stage/staging/')

# Queries and DuckDB

In [ ]:
import duckdb
duckdb.sql("""                                                                                                                   
INSTALL spatial;                                                                                                                     
LOAD spatial;                                                                                                                        
""")


In [ ]:
#duckdb.sql("SELECT * FROM duckdb_settings()")
duckdb.sql("SHOW TABLES")

In [ ]:
duckdb.sql("SET s3_region='us-west-2'; DROP TABLE IF EXISTS addresses; CREATE TABLE addresses AS SELECT  * EXCLUDE geometry, ST_AsGeoJSON(ST_GeomFromWKB(geometry)) as geom FROM   read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=addresses/type=*/*', filename=true, hive_partitioning=1) WHERE  bbox.xmin > -123.85333388805361  AND bbox.xmax < -123.67763908625702  AND bbox.ymin > 39.53857292439969  AND bbox.ymax < 39.89595306214338  AND street != 'NONE';")

In [ ]:
# (min_x DOUBLE, min_y DOUBLE, ma
obq = """
SET s3_region='us-west-2';

SELECT
  * EXCLUDE geometry, -- ST_GeomFromWKB(geometry) as geom,
FROM
  read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=buildings/type=building/*', filename=true, hive_partitioning=1)
WHERE
  bbox.xmin > {west}
  AND bbox.xmax < {east}
  AND bbox.ymin > {south}
  AND bbox.ymax < {north}
;
""".format(**ac['dataswales'][0]['geometry']['bbox'])
address_query

In [ ]:
# (min_x DOUBLE, min_y DOUBLE, ma
address_query = """
SET s3_region='us-west-2';
DROP TABLE IF EXISTS adds;
CREATE TABLE adds AS 
SELECT
  * EXCLUDE geometry, -- ST_GeomFromWKB(geometry) as geom,
  ST_AsGeoJSON(ST_MakeEnvelope(bbox.xmin-0.0002, bbox.ymin-0.0002, bbox.xmax+0.0002, bbox.ymax+0.0002)) as geom
FROM
  read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=addresses/type=*/*', filename=true, hive_partitioning=1)
WHERE
  bbox.xmin > {west}
  AND bbox.xmax < {east}
  AND bbox.ymin > {south}
  AND bbox.ymax < {north}
  AND street != 'NONE';
""".format(**ac['dataswales'][0]['geometry']['bbox'])
address_query

In [ ]:
# (min_x DOUBLE, min_y DOUBLE, ma
roads_query = """
SET s3_region='us-west-2';
DROP TABLE IF EXISTS adds;
CREATE TABLE overture_roads AS 
SELECT
  * EXCLUDE geometry, ST_AsGeoJSON(ST_GeomFromWKB(geometry)) as geometry
FROM
  read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=transportation/type=segment/*', filename=true, hive_partitioning=1)
WHERE
  bbox.xmin > {west}
  AND bbox.xmax < {east}
  AND bbox.ymin > {south}
  AND bbox.ymax < {north}
  
""".format(**ac['dataswales'][0]['geometry']['bbox'])
roads_query

In [ ]:
res = duckdb.sql(obq)

In [ ]:
len(res)

In [ ]:
for row in res.fetchall():
    

In [ ]:
duckdb.sql("""select class,subclass FROM adds WHERE names.primary LIKE '%Usal%';""")

In [ ]:
duckdb.sql("select class, subclass, road_surface[1]['value'] , count(*)  from adds group by (class,subclass, road_surface[1]['value'] ) ORDER BY count(*) DESC;").fetchall()

In [ ]:
duckdb.sql("DESCRIBE adds;")

In [ ]:
annos_path = '/root/data/westport/stage/staging/addresses.geojson'
duckdb.sql("DROP TABLE IF EXISTS annos;")
duckdb.sql(f"CREATE TABLE annos AS SELECT COLUMNS('.*') AS \"annos_\\0\"  FROM ST_Read('{annos_path}');")

feat_path = '/root/data/westport/stage/staging/raw_buildings.geojson'
duckdb.sql("DROP TABLE IF EXISTS features;")
duckdb.sql(f"CREATE TABLE features AS SELECT COLUMNS('.*') AS \"features_\\0\"  FROM ST_Read('{feat_path}');")

In [ ]:
missed_sql="""
SELECT  features.* EXCLUDE features_geom, ST_AsGeoJSON(features_geom) AS geometry                                                
    FROM features LEFT JOIN annos                                                                                                    
    ON ST_Intersects(annos_geom, features_geom)                                                                                      
    WHERE annos_geom IS NULL; 
    """

In [ ]:
mr=duckdb.sql(missed_sql)

In [ ]:
mr.columns

In [ ]:
duckdb.sql("CREATE TABLE buildings AS SELECT * FROM ST_Read('/root/data/westport/stage/staging/buildings.geojson');")

In [ ]:
r=duckdb.sql(
    "SELECT *   FROM adds JOIN buildings ON ST_Intersects(adds.geom, buildings.geom);")

In [ ]:
r.fetchone()

In [ ]:
duckdb.sql(f"DROP TABLE annos")
duckdb.sql(f"CREATE TABLE annos AS SELECT COLUMNS('.*') AS \"annos_\\0\" FROM ST_Read('/root/data/scvfd/stage/ponds_anno/data_20250414_005330.json');")
duckdb.sql(f"DROP TABLE features")
duckdb.sql(f"CREATE TABLE features AS SELECT  COLUMNS('.*') AS \"features_\\0\"  FROM ST_Read('/root/data/scvfd/stage/staging/ponds.geojson');")

In [ ]:
geojoin="""
with foo AS (
    SELECT  *,ST_Distance(ST_Centroid(features_geom), ST_CEntroid(annos_geom)) as dist
    FROM features JOIN annos 
    ON ST_Intersects(annos_geom, features_geom)
    ) 
SELECT b.* EXCLUDE features_geom, ST_AsGeoJSON(b.features_geom) AS geom FROM foo AS b LEFT JOIN foo as c 
ON b.features_geom == c.features_geom AND b.dist < c.dist
WHERE c.dist IS NULL;
"""

In [ ]:
r2=duckdb.sql(geojoin)

In [ ]:
w=r2.fetchall()

In [ ]:
r2.columns

In [ ]:
r2.columns

In [ ]:
r2.columns

In [ ]:
duckdb.sql("describe table features")

In [ ]:
r=duckdb.sql(
    "SELECT * EXCLUDE (annos_geom,features_geom),ST_AsGeoJSON(features_geom) AS geometry  FROM features LEFT JOIN annos ON ST_Intersects(annos_geom, features_geom);")

In [ ]:
res=[dict(zip(r.columns, row)) for row in r.fetchall()]

In [ ]:
res

In [ ]:
fs = geojson.FeatureCollection([])
#dir(fs)

In [ ]:
features = []
for mf in res[:3]:
    f = geojson.Feature()
    for k,v	in ( (k[9:],v) for k,v in mf.items() if k.startswith('features_')):
       
        if k == 'geom':
            f['geometry'] = v
        else:
            f['properties'][k] = v
    features.append(f)
fs = geojson.FeatureCollection(features)

In [ ]:
geojson.dumps(fs, sort_keys=True)

In [ ]:
s=huh[0]['geometry']

In [ ]:
dir(duckdb)

In [ ]:
geojson.loads(s)

In [ ]:
fc = geojson.FeatureCollection()
for f in res:
    fr = geojson.Feature()
    for k,v in fr.items():
    geometry=f['features_geometry'])
    
    del(f['features_geometry'])
    del(f['annos_geometry'])
    f['properties'] = 
    

# WVFD adaptation

In [ ]:
def points_to_bbox(points):
    xset = [p[0] for p in points]
    yset = [p[1] for p in points]
    return {
        "north" : max(yset),
        "south" : min(yset),
        "east": max(xset),
        "west": min(xset),
        #"xset": xset,
        #"yset": yset
    }

roi = json.load(open("westport_runbook.geojson"))
[{'bbox': points_to_bbox(f['geometry']['coordinates'][0]),
    "name": f['properties']['Description'],
    "layers": ["hillshade", "buildings", "creeky", "roads"],
    "caption": f['properties']['Description'],
    "vectors": [], "raster": "",
     "text": f['properties']['Description'] } for f in roi['features']]

In [ ]:
roi['features'][1]

In [ ]:
#roi['features'][1]['geometry']['coordinates']
points_to_bbox(roi['features'][1]['geometry']['coordinates'][0])
points_to_bbox(roi['features'][1]['geometry']['coordinates'][0])

In [ ]:
import geopandas
geo = dc['geometry']['bbox']
geo_tuple = (geo['west'], geo['south'], geo['east'], geo['north'])
gpd = geopandas.read_file("/root/data/WVFD_clippedTA83roads2025_02_04.gpkg",
                        ).to_crs(crs=dc['crs']).clip(geo_tuple)

# Prototyping for tranlations and adaptors

In [ ]:
heli_str = """
39.3342	123.4594	
39.3325	123.4588	
		
39.3444	123.4635	
39.3414	123.4626	
		
39.3681	123.4689	Power Lines overhead
39.3595	123.4708	
39.3829	123.4714	
39.385	123.471	
39.3829	123.4714	
39.3898	123.468	
39.3944	123.4707	
39.3964	123.4709	
39.3944	123.4747	
39.396	123.467	
39.3925	123.4651	
39.7979	123.4617	
39.4074	123.4736	
39.4044	123.4739	
39.3944	123.4707	
39.3925	123.4651	616 ft ABSL
39.4044	123.4739	
39.4022	123.4677	Powerlines South of Road
39.4116	123.4756	
39.4074	123.4736	
39.4214	123.4823	Closed in Winter
39.4237	123.4825	
39.4425	123.4888	Rockport
39.4647	123.4994	
39.4503	123.4909	"""
helis=[l.split(maxsplit=2) for l in heli_str.split("\n")]

In [ ]:
def hack(s):
    deg, rest = s.split('.')
    degs = float(deg)
    badmins = float(rest)/100.0
    mins = float( rest[:2] + '.' + rest[2:])
    #secs = float(rest[2:])
    
    fdeg = degs + mins/60.0 
    #print(f"D: {degs} M: {mins} NOT {badmins} -> {fdeg}")
    return fdeg#+ secs/3600.0

hack('123.471')

In [ ]:
import geojson
def tuples_to_features(tuples):
    features = []
    for t in tuples:
        if not t:
            continue
        print(f"({t[0]}, -{t[1]}) -> (-{hack(t[1])}, {hack(t[0])})")
        f = geojson.Feature(
            geometry=geojson.Point( (-1*hack(t[1]), hack(t[0]) ) ),
            properties = {'name': t[2]} if len(t) > 2 else {}
        )
        features.append(f)
    return geojson.FeatureCollection(features)

def tsv_to_geojson(rawstring, outpath, as_string=False):
    tuples = [l.split(maxsplit=2) for l in rawstring.split("\n")]
    fc = tuples_to_features(tuples)
    if as_string:
        return geojson.dumps(fc)
    else:
        return geojson.dump(fc, open(outpath,"w"))
    

In [ ]:
tsv_to_geojson(heli_str, 'helilandings.geojson', as_string=False)

# Attic

In [ ]:
[(r['bbox']['north'] , r['bbox']['south']) for r in regions]

In [ ]:
c = json.load(open("/root/data/scvfd/staging/creeks.geojson"))
from collections import Counter
Counter([f['properties']['fcode_description'] for f in c['features']])

In [ ]:
ht1="""
            <!DOCTYPE html>

            <html>
            <head>
                <meta charset="utf-8">
                <title>Edit {name}</title>
                <script src='https://unpkg.com/maplibre-gl@3.6.2/dist/maplibre-gl.js'></script>
                <script src="https://unpkg.com/terra-draw@1.0.0-beta.0/dist/terra-draw.umd.js"></script>
                <link href='https://unpkg.com/maplibre-gl@3.6.2/dist/maplibre-gl.css' rel='stylesheet' />

                <style>
                    body {{ margin: 0; padding: 0; }}
                    #map {{ position: absolute; top: 0; bottom: 0; width: 100%; }}
                    #controls {{
                        position: absolute;
                        top: 10px;
                        right: 10px;
                        z-index: 1;
                        background: white;
                        padding: 10px;
                        border: 1px solid #ccc;
                        border-radius: 4px;
                    }}
                    #feature-text {{
                        margin-bottom: 10px;
                        padding: 5px;
                        width: 200px;
                    }}
                    #save-button {{
                        padding: 10px;
                        background: white;
                        border: 1px solid #ccc;
                        border-radius: 4px;
                        cursor: pointer;
                        width: 100%;
                    }}
                    #save-button:hover {{
                        background: #f0f0f0;
                    }}
                </style>
            </head>
            <body>
            <center>
            <h1>Edit Note Layer: {name}</h1>
            <i>Click to draw, double click to end, click "Save" button when done.</i>
            
            <A HREF="../html">cancel</a>
            </center>
            <div id="controls">
                    <input type="text" id="feature-name" placeholder="Enter name" />
                    
                    <input type="radio" id="feature-fcode_description" name="feature-fcode_description" value="Stream/River: Hydrographic Category = Intermittent" />
                    
            <button id="save-button">Save Note</button>
            </div>
            
            <div id="map"></div>
            <script>
            """
ht2="""
                var map = new maplibregl.Map({
  "container": "map",
  "style": {
    "glyphs": "https://fonts.undpgeohub.org/fonts/{fontstack}/{range}.pbf",
    "version": 8,
    "center": [
      -123.90625574273562,
      40.211
    ],
    "zoom": 12,
    "sources": {
      "hillshade": {
        "type": "image",
        "url": "../staging/hillshade.tiff.jpg",
        "coordinates": [
          [
            -123.98,
            40.26
          ],
          [
            -123.83251148547124,
            40.26
          ],
          [
            -123.83251148547124,
            40.162
          ],
          [
            -123.98,
            40.162
          ]
        ]
      },
      "roads": {
        "type": "geojson",
        "data": "/scvfd/staging/roads.geojson"
      },
      "buildings": {
        "type": "geojson",
        "data": "/scvfd/staging/buildings.geojson"
      }
    },
    "layers": [
      {
        "id": "hillshade-layer",
        "type": "raster",
        "source": "hillshade",
        "paint": {
          "raster-opacity": 1,
          "raster-contrast": 0.3
        }
      },
      {
        "id": "roads-layer",
        "type": "line",
        "source": "roads",
        "text-field": [
          "get",
          "name"
        ],
        "paint": {
          "line-color": "rgb(0, 0, 0)",
          "line-width": [
            "match",
            [
              "get",
              "STREETTYPE"
            ],
            "RD",
            5,
            "DR",
            3,
            2
          ]
        }
      },
      {
        "id": "roads-label-layer",
        "type": "symbol",
        "source": "roads",
        "layout": {
          "symbol-placement": "line",
          "text-offset": [
            0,
            2
          ],
          "text-font": [
            "Open Sans Regular"
          ],
          "text-field": [
            "get",
            "STREETNAME"
          ],
          "text-size": 20
        }
      },
      {
        "id": "buildings-layer",
        "type": "fill",
        "source": "buildings",
        "text-field": [
          "get",
          "name"
        ],
        "paint": {
          "fill-color": "rgb(200, 50, 50)"
        }
      }
    ]
  }
});
"""
ht4="""
            const td = new terraDraw.TerraDraw({{
                adapter: new terraDraw.TerraDrawMapLibreGLAdapter({{
                map: map,
                lib: maplibregl,
                }}),
                modes: [new terraDraw.TerraDrawLineStringMode()],
            }});

            // Initialize Terra Draw
            td.start();

            td.setMode("{modestring}");

            // Add save button functionality
            document.getElementById('save-button').addEventListener('click', function() {{
                const features = td.getSnapshot();
                
                {controls_string}
                // const featureText = document.getElementById('feature-text').value;
                // const vecWidth = document.getElementById('{width_att}').value;
                // const features = td.getFeatures();
                
                // Add name to each feature's properties
                features.forEach(feature => {{
                    if (!feature.properties) {{
                        feature.properties = {{}};
                    }}
                    feature.properties.text = featureText;
                    feature.properties.{width_att} = vecWidth;
                
                }});
                const geojson = {{
                    "type": "FeatureCollection",
                    "layer": "{ea['name']}",
                    "features": features
                    }};
                
                for(i = 0; i < features.length; i++){{
                    var xmlhttp = new XMLHttpRequest();   // new HttpRequest instance 
                    xmlhttp.open("POST", 'http://fireatlas.org:9998/store');
                    xmlhttp.setRequestHeader("Content-Type", "application/json");
                    var geojson_data = JSON.stringify({{"data":geojson}});
                    alert(geojson_data);
                    xmlhttp.send(geojson_data);
                    }};
                xmlhttp.onreadystatechange = function() {{
                if (xmlhttp.readyState == 4 && xmlhttp.status == 200) {{
                    alert('upload successful!');
                }} else if (xmlhttp.readyState == 4 && xmlhttp.status !== 200){{
                    alert('looks like something went wrong');
                }}
            }} }} );
            

            </script></body></html>
"""

In [ ]:
grass = '/usr/bin/grass'
sys.path.insert(
    0,subprocess.check_output([grass, "--config", "python_path"], text=True).strip())

my_env = os.environ.copy()
import grass.jupyter as gj
GRASS_LOC = dc['name']
# GRASS_LOC = GRASS_LOC_NAME + datetime.datetime.now().strftime("%I:%M%p_%B-%d-%Y")
session = gj.init("~/grassdata", GRASS_LOC, "PERMANENT")

my_env["PYTHONPATH"] = f"/usr/lib/grass83/etc/python:{my_env['PATH']}"

# TODO: ???
import grass.script as gs
    

In [ ]:
ac['outlets']['geopackage']['layers'] = []

atlas.materialize_outlet(ac, dc, ac['outlets']['geopackage'])

In [ ]:
atlas.grass_location_initialize("scvfd")

In [ ]:
atlas.build_map("scvfd")

In [ ]:
atlas.map_image("scvfd")

In [ ]:
g.crs

g2=g.to_crs("EPSG:4269")

g.explore()

# g.clip(geo_polygon).explore()
g.clip(g3).explore()

import shapely
geo = dc['geometry']['bbox']
geo_polygon = shapely.box(geo['west'], geo['south'], geo['north'], geo['east'])

from geojson import Feature, FeatureCollection,Point, Polygon
import geopandas as gpd

[(geo[x], geo[y]) for x,y in [
                ('west', 'north'), ('east', 'north'), ('east', 'south'), ('west', 'south')]
                 ]

pts=[(geo[x], geo[y]) for x,y in [
                ('west', 'north'), ('east', 'north'), ('east', 'south'), ('west', 'south')]
                 ]
fc=FeatureCollection([Feature(geometry=Polygon([pts]))])
fc

#poly = FeatureCollection([Feature(geometry=Polygon(pts))])
g3=gpd.GeoDataFrame.from_features(fc)

g3.set_crs("EPSG:4326").explore()
# EPSG:4326

g3.set_crs("EPSG:4269").explore()

del(g['quadkey'])

g.to_file("test2.geojson", driver="GeoJSON",crs='EPSG:4269', engine="fiona")

In [ ]:
ac['outlets'].keys()

In [ ]:
ac['eddies']

In [ ]:
bb=dc['geometry']['bbox']
w=(bb['east'] - bb['west'])/4.0


In [ ]:
bb['north'] - 3.0* w

In [ ]:
import matplotlib as mpl
cmap = mpl.colormaps['gist_gray']

In [ ]:
#myc=cmap.copy()

In [ ]:
import numpy as np
f=lambda x: (9.0+x)/10.0
myc = mpl.colors.ListedColormap([[f(r), f(g), f(b), a] for r,g,b,a in cmap(np.linspace(0.0,1.0, 20))])


In [ ]:
myc

In [ ]:
cmap

In [ ]:
dir(cmap)

In [ ]:
math.radians

In [ ]:
def ll2xyz(lat, long,z):
    n = 2**z
    r = math.radians(lat)
    lts = math.log(math.tan(r) + 1.0/math.cos(r))
    x = n*(0.5 + long/360)
    y = n*(1 - lts/math.pi) / 2
    return math.floor(x),math.floor(y),z

In [ ]:
ll2xyz(40.25, -123.92, 12)

In [ ]:
from fastapi import FastAPI

In [ ]:
a = FastAPI()

In [ ]:
dir(a)

In [ ]:
import fastapi
dir(fastapi.logger.logger)
#fastapi.logger.logger.handlers

fastapi.logger.logger.setLevel(0)

https://fireatlas.org/{x}/{y}/{z}.png

https://geojson.io/#new&map=12/40.228/-123.8937

In [ ]:
fastapi.logger.logger.error("HI")

In [ ]:
import logging
dir(logging)